In [1]:
from pyspark.sql.functions import col,substring
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql import functions as F, SparkSession
from pyspark.sql.window import Window
import pandas as pd

In [2]:
df=spark.read.orc('/apps/hive/warehouse/tntqc2_454.db/role_resource/ds*')

In [3]:
df1=df.select('game_id','event_id','server_id','createtime_local','time','account','role_id','role_name','diamond','black_diamond','money').withColumn(
  "rank", dense_rank().over(Window.partitionBy('account','role_id').orderBy(desc("createtime_local"))))

In [4]:
df1.show(truncate=False)

+-------+--------+----------+-------------------+-------------------+---------------------------+-------------+----------------+-------+-------------+------+----+
|game_id|event_id|server_id |createtime_local   |time               |account                    |role_id      |role_name       |diamond|black_diamond|money |rank|
+-------+--------+----------+-------------------+-------------------+---------------------------+-------------+----------------+-------+-------------+------+----+
|2002012|794081  |2772310067|2021-01-29 00:03:28|2021-01-29 00:03:28|0060411_1716541037649821696|3710851753746|Chiết Bành Hoạch|0      |488          |939838|1   |
|2002012|724759  |2772310067|2021-01-28 01:19:21|2021-01-28 01:19:21|0060411_1716541037649821696|3710851753746|Chiết Bành Hoạch|0      |488          |939838|2   |
|2002012|724758  |2772310067|2021-01-28 01:19:17|2021-01-28 01:19:17|0060411_1716541037649821696|3710851753746|Chiết Bành Hoạch|0      |488          |939838|3   |
|2002012|724753  |2772

In [4]:
df2=df1.filter(df1.rank=='1').drop('money').dropDuplicates()

In [5]:
array=['2768420001','2770420001','2770420002','2771420001','2771440001']

In [6]:
df2=df2.filter(df.server_id.isin (array) == False)

In [8]:
df2.show(10)

+-------+--------+----------+-------------------+-------------------+--------------------+-------------+----------------+-------+-------------+----+
|game_id|event_id| server_id|   createtime_local|               time|             account|      role_id|       role_name|diamond|black_diamond|rank|
+-------+--------+----------+-------------------+-------------------+--------------------+-------------+----------------+-------+-------------+----+
|2002012|  794081|2772310067|2021-01-29 00:03:28|2021-01-29 00:03:28|0060411_171654103...|3710851753746|Chiết Bành Hoạch|      0|          488|   1|
|2002012|  678497|2772310100|2021-03-06 09:27:01|2021-03-06 09:27:01|0060411_171654103...|7352984023253|      Di Chu Bái|      0|          100|   1|
|2002012| 2329878|2772310130|2021-03-31 00:28:32|2021-03-31 00:28:32|0060411_171654103...|9277129371502|     Nhữ Hạ Liêm|      0|          100|   1|
|2002012|   13572|2772310002|2020-10-19 10:07:29|               null|0060411_173681075...| 824633730835|  

In [9]:
 df2.count()

1022839

In [7]:
df2=df2.drop('rank','time')

In [11]:
df2.show()

+-------+--------+----------+-------------------+--------------------+-------------+----------------+-------+-------------+
|game_id|event_id| server_id|   createtime_local|             account|      role_id|       role_name|diamond|black_diamond|
+-------+--------+----------+-------------------+--------------------+-------------+----------------+-------+-------------+
|2002012|  794081|2772310067|2021-01-29 00:03:28|0060411_171654103...|3710851753746|Chiết Bành Hoạch|      0|          488|
|2002012|  678497|2772310100|2021-03-06 09:27:01|0060411_171654103...|7352984023253|      Di Chu Bái|      0|          100|
|2002012| 2329878|2772310130|2021-03-31 00:28:32|0060411_171654103...|9277129371502|     Nhữ Hạ Liêm|      0|          100|
|2002012|   13572|2772310002|2020-10-19 10:07:29|0060411_173681075...| 824633730835|           TN3Q1|      0|        11347|
|2002012|   13572|2772310002|2020-10-19 10:07:29|0060411_173681075...| 824633730835|           TN3Q1|      0|        11497|
|2002012

In [47]:
df3=df2.withColumn('last_log_date', df2.createtime_local.substr(1, 10)).drop('createtime_local','game_id','event_id','role_name')

In [13]:
df3.show(truncate=False)

+----------+---------------------------+-------------+-------+-------------+----------+
|server_id |account                    |role_id      |diamond|black_diamond|ds        |
+----------+---------------------------+-------------+-------+-------------+----------+
|2772310067|0060411_1716541037649821696|3710851753746|0      |488          |2021-01-29|
|2772310100|0060411_1716541037649821696|7352984023253|0      |100          |2021-03-06|
|2772310130|0060411_1716541037649821696|9277129371502|0      |100          |2021-03-31|
|2772310002|0060411_1736810755623428096|824633730835 |0      |11347        |2020-10-19|
|2772310002|0060411_1736810755623428096|824633730835 |0      |11497        |2020-10-19|
|2772310002|0060411_1740377687341604864|824633730832 |0      |100          |2020-11-02|
|2772310006|0060411_1766740149418295296|549755828759 |0      |1200         |2020-11-06|
|2772310103|0060411_1766744728532910080|7146825592785|0      |2348         |2021-02-10|
|2772310003|0060411_176674619467

In [30]:
df3.count()

974173

In [48]:
df3=df3.filter(df3.last_log_date > '2020-11-05').distinct()

In [29]:
df3.show(truncate=False)

+----------+---------------------------+--------------+-------+-------------+----------+
|server_id |account                    |role_id       |diamond|black_diamond|ds        |
+----------+---------------------------+--------------+-------+-------------+----------+
|2772310067|0060411_1716541037649821696|3710851753746 |0      |488          |2021-01-29|
|2772310100|0060411_1716541037649821696|7352984023253 |0      |100          |2021-03-06|
|2772310130|0060411_1716541037649821696|9277129371502 |0      |100          |2021-03-31|
|2772310006|0060411_1766740149418295296|549755828759  |0      |1200         |2020-11-06|
|2772310103|0060411_1766744728532910080|7146825592785 |0      |2348         |2021-02-10|
|2772310002|0060411_1766753143498153984|824633734754  |0      |768          |2020-11-06|
|2772310031|0060411_1766754472354086912|2061584317427 |0      |6504         |2021-01-09|
|2772310026|0060411_1766755663544811520|2405181704874 |0      |1280         |2021-04-05|
|2772310004|0060411_1

In [49]:
df4=df3.select('server_id','account','role_id','black_diamond','last_log_date').distinct()
df4.show(truncate=False)

+----------+---------------------------+--------------+-------------+-------------+
|server_id |account                    |role_id       |black_diamond|last_log_date|
+----------+---------------------------+--------------+-------------+-------------+
|2772310067|0060411_1716541037649821696|3710851753746 |488          |2021-01-29   |
|2772310100|0060411_1716541037649821696|7352984023253 |100          |2021-03-06   |
|2772310130|0060411_1716541037649821696|9277129371502 |100          |2021-03-31   |
|2772310006|0060411_1766740149418295296|549755828759  |1200         |2020-11-06   |
|2772310103|0060411_1766744728532910080|7146825592785 |2348         |2021-02-10   |
|2772310002|0060411_1766753143498153984|824633734754  |768          |2020-11-06   |
|2772310031|0060411_1766754472354086912|2061584317427 |6504         |2021-01-09   |
|2772310026|0060411_1766755663544811520|2405181704874 |1280         |2021-04-05   |
|2772310004|0060411_1766759369539788800|687194788580  |2634         |2020-11

In [50]:
df4.toPandas().to_csv('tntqc2_all_roleid.csv')

In [31]:
df4.groupBy("role_id").agg(countDistinct("account").alias("count")).orderBy(desc("count")).show(5)

+--------------+-----+
|       role_id|count|
+--------------+-----+
|  343597405285|    1|
|  618475304225|    1|
|10995116288876|    1|
|  481036349630|    1|
|  481036359370|    1|
+--------------+-----+
only showing top 5 rows



In [32]:
df4.groupBy("role_id").agg(countDistinct("account").alias("count")).filter("count > 1").show()

+-------+-----+
|role_id|count|
+-------+-----+
+-------+-----+



In [43]:
# a.toPandas().to_csv('tntqc2_dup_roleid.csv')

In [41]:
a=df4.select('server_id','role_id')

In [51]:
df=spark.read.option("sep","\\t").csv('/projects/cts-parser-upofb/warehouse/banned_user_full',header=True)

In [52]:
da=df.filter(df.product_alias=='TNTQC2')

In [53]:
da.show(truncate=False)

+------------+-------------+-------------+--------+---------------------------+-------------+-------+-------------------+--------+--------------+
|product_code|dept         |product_alias|priority|user_id                    |role_id      |open_id|banned_date        |timezone|account_id    |
+------------+-------------+-------------+--------+---------------------------+-------------+-------+-------------------+--------+--------------+
|454         |Game Studio 9|TNTQC2       |1st     |0060411_1775157815187546112|3092376470800|null   |2020-11-18 14:10:00|GMT+7   |HK Vô Ngã     |
|454         |Game Studio 9|TNTQC2       |1st     |0060411_1778044909232791552|2817498562194|null   |2020-11-18 13:49:00|GMT+7   |ZzWindKjdzZ   |
|454         |Game Studio 9|TNTQC2       |1st     |0060411_1777265829000454144|2886218036929|null   |2020-11-18 14:28:00|GMT+7   |AnhBa         |
|454         |Game Studio 9|TNTQC2       |1st     |0060411_1776732236062621696|3298534915093|null   |2020-11-18 14:08:00|GMT

In [37]:
da.count()

87

In [54]:
joined_ban=df4.join(da, on= [df3.role_id==da.role_id], how='left').dropDuplicates()

In [55]:
joined_ban.printSchema()

root
 |-- server_id: string (nullable = true)
 |-- account: string (nullable = true)
 |-- role_id: string (nullable = true)
 |-- black_diamond: long (nullable = true)
 |-- last_log_date: string (nullable = true)
 |-- product_code: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- product_alias: string (nullable = true)
 |-- priority: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- role_id: string (nullable = true)
 |-- open_id: string (nullable = true)
 |-- banned_date: string (nullable = true)
 |-- timezone: string (nullable = true)
 |-- account_id: string (nullable = true)



In [57]:
joined_ban=joined_ban.select('server_id','account',df4.role_id,'black_diamond','last_log_date')

In [58]:
joined_ban.show(truncate=False)

+----------+---------------------------+--------------+-------------+-------------+
|server_id |account                    |role_id       |black_diamond|last_log_date|
+----------+---------------------------+--------------+-------------+-------------+
|2772310131|0060411_1858390203228454912|10033043613552|8007         |2021-05-30   |
|2772310131|0060411_1858413036457631744|10033043613650|1310         |2021-03-08   |
|2772310131|0060411_1810588779174633472|10033043613827|70           |2021-03-19   |
|2772310131|0060411_1858687334443327488|10033043614567|100          |2021-03-08   |
|2772310131|0060411_1858709718537568256|10033043614666|0            |2021-03-08   |
|2772310131|0060411_1858711046823313408|10033043614669|764          |2021-03-24   |
|2772310131|0060411_1858717344327499776|10033043614694|100          |2021-03-13   |
|2772310131|0060411_1859102247275929600|10033043615245|4679         |2021-03-13   |
|2772310160|0060411_1879421643777056768|10101763090279|100          |2021-04

In [88]:
joined_ban.count()

974172

In [59]:
joined_ban.toPandas().to_csv('tntqc2_joinedban.csv')

In [60]:
df=spark.read.option("sep","\\t").csv('/projects/cts-parser-upofb/warehouse/banned_user_full',header=True)

In [61]:
db=df.filter(df.product_alias=='TNTQC2')

In [62]:
db.show(truncate=False)

+------------+-------------+-------------+--------+---------------------------+-------------+-------+-------------------+--------+--------------+
|product_code|dept         |product_alias|priority|user_id                    |role_id      |open_id|banned_date        |timezone|account_id    |
+------------+-------------+-------------+--------+---------------------------+-------------+-------+-------------------+--------+--------------+
|454         |Game Studio 9|TNTQC2       |1st     |0060411_1775157815187546112|3092376470800|null   |2020-11-18 14:10:00|GMT+7   |HK Vô Ngã     |
|454         |Game Studio 9|TNTQC2       |1st     |0060411_1778044909232791552|2817498562194|null   |2020-11-18 13:49:00|GMT+7   |ZzWindKjdzZ   |
|454         |Game Studio 9|TNTQC2       |1st     |0060411_1777265829000454144|2886218036929|null   |2020-11-18 14:28:00|GMT+7   |AnhBa         |
|454         |Game Studio 9|TNTQC2       |1st     |0060411_1776732236062621696|3298534915093|null   |2020-11-18 14:08:00|GMT

In [63]:
ban_churn=joined_ban.join(db, 'role_id', how='full').dropDuplicates()

In [65]:
ban_churn=ban_churn.select('server_id','account','role_id','black_diamond','last_log_date','banned_date')

In [66]:
ban_churn.show(truncate=False)

+----------+---------------------------+--------------+-------------+-------------+-----------+
|server_id |account                    |role_id       |black_diamond|last_log_date|banned_date|
+----------+---------------------------+--------------+-------------+-------------+-----------+
|2772310185|0060411_1897716938491002880|12506944776689|1466         |2021-05-01   |null       |
|2772310039|0060411_1791111830500769792|1511828501974 |46           |2020-12-10   |null       |
|2772310038|0060411_1790983106111438848|1580547975945 |200          |2020-12-05   |null       |
|2772310032|0060411_1786945820084609024|1992864842906 |113          |2020-12-02   |null       |
|2772310028|0060411_1783105641003556864|2267742742700 |1500         |2020-11-25   |null       |
|2772310027|0060411_1783008073556656128|2336462219193 |1100         |2020-11-24   |null       |
|2772310026|0060411_1796516839518339072|2405181705830 |2881         |2021-05-25   |null       |
|2772310022|0060411_1779623653386952704|

In [32]:
ban_churn.filter(ban_churn.account.isin(['0060411_1776732236062621696','0060411_1769613309423853568','0060411_1775111952477945856'])).show(truncate=False)

+-------------+----------+---------------------------+-------+-------------+----------+------------+-------------+-------------+--------+---------------------------+-------------+-------+-------------------+--------+-------------+------------+-------------+-------------+--------+---------------------------+-------+-------------------+--------+-------------+
|role_id      |server_id |account                    |diamond|black_diamond|ds        |product_code|dept         |product_alias|priority|user_id                    |role_id      |open_id|banned_date        |timezone|account_id   |product_code|dept         |product_alias|priority|user_id                    |open_id|banned_date        |timezone|account_id   |
+-------------+----------+---------------------------+-------+-------------+----------+------------+-------------+-------------+--------+---------------------------+-------------+-------+-------------------+--------+-------------+------------+-------------+-------------+--------+

In [67]:
ban_churn.toPandas().to_csv('tntqc2_banchurn.csv')